This notebook is to clean the twitter data/join on the NBA game data so that it can be used for analysis. Also joins the mavs game database based on date to tweets on that date.

In [1]:
import pandas as pd
import datetime as datetime
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import tqdm
from nltk.tokenize import word_tokenize
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\micha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\micha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [58]:
mavs_df = pd.read_csv('Datasets\mavs_tweets_all.csv')
mavs_df


,Datetime,tweet_id,username,text,verified,retweet_count,like_count
0,2022-02-09 23:58:50+00:00,1491562265316773888,TheMontyShow,@603live I have been told the Mavs and Bulls a...,False,0,1
1,2022-02-09 23:58:30+00:00,1491562180872974336,neilbobnuss,@SpotOn96057010 @RGutman_ @Mavs_FFL Your first...,False,0,1
2,2022-02-09 23:58:30+00:00,1491562180281577480,tac234,"Trade for Hield, Nurkic, you know players that...",False,0,1
3,2022-02-09 23:58:25+00:00,1491562158374723585,VeritasetfidePF,"@cazefontoura Ja hj ,ninguém sabe quem é. \nSo...",False,0,4
4,2022-02-09 23:58:12+00:00,1491562107258687490,VCPHoops,"Mavs Post Game Interview: Jason Kidd, Trey Bur...",False,0,0
...,...,...,...,...,...,...,...
176118,2021-10-20 00:02:19+00:00,1450613312778031108,ChrisGallegosNV,@MensaNinja @Mavs_FFL I’m not saying throw the...,False,0,0
176119,2021-10-20 00:01:13+00:00,1450613037568872454,MensaNinja,@dallassportzone @isaactweeting @All_Things_Ma...,False,0,0
176120,2021-10-20 00:01:02+00:00,1450612991649632262,ZachPrell,"@NickGamerForceJ I sure hope so, just want to ...",False,0,1
176121,2021-10-20 00:00:54+00:00,1450612957193261056,Aaron_Del24,@KillSwitchTD92 @rawljul Jazz\nSuns\nLakers\nM...,False,0,0


In [81]:
tweet_text = mavs_df['text']

Steps to preprocessing the tweet text:
1. Removing @'s
2. Removing links
3. Separating hashtags
4. Removing stop words

In [90]:
#Cleaning Functions

def remove_ats(tw): 
    '''
    Removes @s from a tweet. 
    Improvements: 
       2. if at end of tweet still delete the handle
    '''
    if '@' not in tw: 
        return tw
    
    new_tw = tw
    while '@' in new_tw:
        index_of_at = new_tw.index('@')
        search_range = new_tw[index_of_at:]
        for i, char in enumerate(search_range): 
            if char == ' ': 
                end_index_at = i
                break
            elif i == (len(search_range) - 1): 
                end_index_at = i + 1
                break    
        delete_text = new_tw[index_of_at:end_index_at + index_of_at]
        new_tw = new_tw.replace(delete_text, '')

    new_tw = new_tw.strip()
    return new_tw

def remove_links(tw): 
    '''
    Removes https links from tweets. 
    '''
    if 'https' not in tw: 
        return tw
    
    new_tw = tw
    while 'https' in new_tw:
        index_of_http = new_tw.find('https')
        search_range = new_tw[index_of_http:]
        for i, char in enumerate(search_range): 
            if i == (len(search_range) - 1): 
                end_index_http = i + 1
                break
            elif char == ' ': 
                end_index_http = i
                break
        
        delete_text = new_tw[index_of_http: end_index_http + index_of_http]   
        new_tw = new_tw.replace(delete_text, '')
    new_tw = new_tw.strip()
    return new_tw

def separate_hashtags(tw): 
    '''
    Finds and seperates hashtags. 
    Improvements: 
       1. cases where no words in the hashtag are capitalized
       2. when hashtag words are in dictionary
    '''
    if '#' not in tw: 
        return tw
    
    new_tw = tw
    while '#' in new_tw:
        index_of_hash = new_tw.find('#')
        search_range = new_tw[index_of_hash:]
        for i, char in enumerate(search_range): 
            if char == ' ': 
                end_index_hash = i
                break 
            elif i == (len(search_range) - 1): 
                end_index_hash = i + 1
                break
        
        new_hash_text = ''
        text_range = new_tw[index_of_hash:end_index_hash + index_of_hash]
        for char in text_range: 
            if char.islower():
                new_hash_text += char
            else: 
                new_hash_text += ' '
                new_hash_text += char.lower()
        
        new_hash_text = new_hash_text.replace('#', '')
        delete_text = new_tw[index_of_hash:end_index_hash + index_of_hash]
        new_tw = new_tw.replace(delete_text, new_hash_text)
    
    new_tw = new_tw.strip()
    return new_tw

def remove_stopwords(tw):
    '''
    Removes stopwords from the tweet text.
    Improvements:
    1. Abbreviations can leave random letters
    2. Some words in the stopwords list may be useful (over, under)
    '''
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tw)
    
    new_tw = []
    for word in word_tokens:
        if word not in stop_words:
            new_tw.append(word)
    
    return(" ".join(new_tw))

In [91]:
#Using the above the functions to perform text preprocessing

tweet_text = mavs_df['text']

cleaned_tws = []
for tweet in tqdm.tqdm(tweet_text): 
    stage_one = remove_ats(tweet)
    stage_two = remove_links(stage_one)
    stage_three = separate_hashtags(stage_two)
    final_tweet = remove_stopwords(stage_three)
    cleaned_tws.append(final_tweet)

100%|██████████| 176123/176123 [01:11<00:00, 2461.44it/s]


Next, using the list output as the text column in a new dataframe.

In [95]:
mavs_df_textprocessed = mavs_df
mavs_df_textprocessed['text'] = cleaned_tws
mavs_df_textprocessed.to_csv('mavs_df_textprocessed.csv')

Joining the mavs game database to this database

In [8]:
#Fixing the date formatting in the game data
game_df = pd.read_csv('Datasets\Mavs_game_data.csv')
gameDates = game_df['Date']
gameDates = pd.to_datetime(gameDates)

game_df['Date'] = gameDates
game_df.tail()

,Unnamed: 0,Date,Opposing Team,Win or Loss,Score,Record,Leading Scorer-Winner,Leading Scorer Points,Leading Scorer Loss,Leading Scorer Points.1
52,52,2022-02-04,Philadelphia,W,107-98,30-23,luka-doncic,(33),joel-embiid,(27)
53,53,2022-02-06,Atlanta,W,103-94,31-23,jalen-brunson,(22),john-collins,(22)
54,54,2022-02-08,Detroit,W,116-86,32-23,luka-doncic,(33),hamidou-diallo,(18)
55,55,2022-02-10,L.A. Clippers,W,112-105,33-23,luka-doncic,(51),marcus-morris,(21)
56,56,2022-02-12,L.A. Clippers,L,99-97,33-24,reggie-jackson,(24),luka-doncic,(45)


In [10]:
#Formatting tweet date
tweets_df = pd.read_csv('Datasets\mavs_tweets_textprocessed.csv')
dates_list = []
for day in tweets_df['Datetime']:
    dates_list.append(str(day)[:10])
tweets_df['Date'] = dates_list
tweets_df['Date'] = pd.to_datetime(tweets_df['Date'])
tweets_df.head()


,Unnamed: 0,Datetime,tweet_id,username,text,verified,retweet_count,like_count,Date
0,0,2022-02-09 23:58:50+00:00,1491562265316773888,TheMontyShow,I told Mavs Bulls likeliest destination Collin...,False,0,1,2022-02-09
1,1,2022-02-09 23:58:30+00:00,1491562180872974336,neilbobnuss,Your first counter offer gets laughed room haha,False,0,1,2022-02-09
2,2,2022-02-09 23:58:30+00:00,1491562180281577480,tac234,"Trade Hield , Nurkic , know players actually h...",False,0,1,2022-02-09
3,3,2022-02-09 23:58:25+00:00,1491562158374723585,VeritasetfidePF,"Ja hj , ninguém sabe quem é . So aqui na rede ...",False,0,4,2022-02-09
4,4,2022-02-09 23:58:12+00:00,1491562107258687490,VCPHoops,"Mavs Post Game Interview : Jason Kidd , Trey B...",False,0,0,2022-02-09


In [11]:
#Joining the two dataframes on the date
gameTweets = pd.merge(tweets_df, game_df, how='left', left_on='Date', right_on='Date')
gameTweets.head()

,Unnamed: 0_x,Datetime,tweet_id,username,text,verified,retweet_count,like_count,Date,Unnamed: 0_y,Opposing Team,Win or Loss,Score,Record,Leading Scorer-Winner,Leading Scorer Points,Leading Scorer Loss,Leading Scorer Points.1
0,0,2022-02-09 23:58:50+00:00,1491562265316773888,TheMontyShow,I told Mavs Bulls likeliest destination Collin...,False,0,1,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2022-02-09 23:58:30+00:00,1491562180872974336,neilbobnuss,Your first counter offer gets laughed room haha,False,0,1,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2022-02-09 23:58:30+00:00,1491562180281577480,tac234,"Trade Hield , Nurkic , know players actually h...",False,0,1,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2022-02-09 23:58:25+00:00,1491562158374723585,VeritasetfidePF,"Ja hj , ninguém sabe quem é . So aqui na rede ...",False,0,4,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2022-02-09 23:58:12+00:00,1491562107258687490,VCPHoops,"Mavs Post Game Interview : Jason Kidd , Trey B...",False,0,0,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#Additional cleaning of the joined tables
gameTweets = gameTweets.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'])
gameTweets.head()

,Datetime,tweet_id,username,text,verified,retweet_count,like_count,Date,Opposing Team,Win or Loss,Score,Record,Leading Scorer-Winner,Leading Scorer Points,Leading Scorer Loss,Leading Scorer Points.1
0,2022-02-09 23:58:50+00:00,1491562265316773888,TheMontyShow,I told Mavs Bulls likeliest destination Collin...,False,0,1,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-02-09 23:58:30+00:00,1491562180872974336,neilbobnuss,Your first counter offer gets laughed room haha,False,0,1,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-02-09 23:58:30+00:00,1491562180281577480,tac234,"Trade Hield , Nurkic , know players actually h...",False,0,1,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-02-09 23:58:25+00:00,1491562158374723585,VeritasetfidePF,"Ja hj , ninguém sabe quem é . So aqui na rede ...",False,0,4,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-02-09 23:58:12+00:00,1491562107258687490,VCPHoops,"Mavs Post Game Interview : Jason Kidd , Trey B...",False,0,0,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#Checking the merge worked properly on a game day
gameDay = gameTweets.loc[gameTweets['Date'] == '2021-10-26']
gameDay

,Datetime,tweet_id,username,text,verified,retweet_count,like_count,Date,Opposing Team,Win or Loss,Score,Record,Leading Scorer-Winner,Leading Scorer Points,Leading Scorer Loss,Leading Scorer Points.1
1256,2021-10-26 23:59:00+00:00,1453149194319843329,upandundertv,"Ingame pictures please 🙏 , lot us like high qu...",False,0,0,2021-10-26,Houston,W,116-106,2-1,luka-doncic,(26),eric-gordon,(22)
1257,2021-10-26 23:58:33+00:00,1453149081270984707,MadPricee,No thoughts vibes,False,1,9,2021-10-26,Houston,W,116-106,2-1,luka-doncic,(26),eric-gordon,(22)
1258,2021-10-26 23:58:29+00:00,1453149061310304259,RedArmyOmaha,Sadly Mavs pizza going movies make list,False,0,7,2021-10-26,Houston,W,116-106,2-1,luka-doncic,(26),eric-gordon,(22)
1259,2021-10-26 23:57:39+00:00,1453148855269220355,druidandivote,I ’ know anyone even receive Bally Sports . Ni...,False,0,0,2021-10-26,Houston,W,116-106,2-1,luka-doncic,(26),eric-gordon,(22)
1260,2021-10-26 23:57:33+00:00,1453148829482618895,DanielMrClutch,Previa de Houston ante los Mavs ( NBA ) vía,False,1,1,2021-10-26,Houston,W,116-106,2-1,luka-doncic,(26),eric-gordon,(22)
1261,2021-10-26 23:57:32+00:00,1453148825762291712,treyjustice,Can ’ watch mavs cause Bally sports ass,False,0,0,2021-10-26,Houston,W,116-106,2-1,luka-doncic,(26),eric-gordon,(22)
1262,2021-10-26 23:57:24+00:00,1453148791788478465,jtuck151,Bruh I got Clemson Saturday I ’ sure I ’ ready...,False,0,1,2021-10-26,Houston,W,116-106,2-1,luka-doncic,(26),eric-gordon,(22)
1263,2021-10-26 23:57:21+00:00,1453148776168837122,ogtonyj3000,I need Mavs & amp ; Lakers good game tonight . 🙏🏾,False,0,2,2021-10-26,Houston,W,116-106,2-1,luka-doncic,(26),eric-gordon,(22)
1264,2021-10-26 23:57:08+00:00,1453148722616053761,local_mariah,Got offered tick mavs game tonight,False,0,0,2021-10-26,Houston,W,116-106,2-1,luka-doncic,(26),eric-gordon,(22)
1265,2021-10-26 23:56:57+00:00,1453148678097670145,llcooljoness,go mavs go braves go sports,False,0,2,2021-10-26,Houston,W,116-106,2-1,luka-doncic,(26),eric-gordon,(22)


In [15]:
#Saving the table to csv
gameTweets.to_csv('Datasets\mavs_games_tweets_merged.csv')